## 1 数据预处理

In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import os

In [2]:
# 载入原始数据
def load_data(base_path):
    paths = os.listdir(base_path)
    result = []
    for path in paths:
        with open(os.path.join(base_path, path), 'r', encoding='utf-8') as f:
            result.append(f.readline())
    return result

# 读入数据并转化为datasets.Dataset
def get_dataset(base_path):
		# 为了展示方便，这里只取前3个数据，真实使用需要删掉切片操作
    pos_data = load_data(os.path.join(base_path, 'pos'))
    neg_data = load_data(os.path.join(base_path, 'neg'))
    
		# 列表合并
    texts = pos_data + neg_data
		# 生成标签，其中使用 '1.' 和 '0.' 是因为需要转化为浮点数，要不然模型训练时会报错
    labels = [[1., 0.]]*len(pos_data) + [[0., 1.]] * len(neg_data)
    dataset = Dataset.from_dict({'texts':texts, 'labels':labels})
    return dataset

# 加载数据
train_dataset = get_dataset('../aclImdb/train/')
test_dataset = get_dataset('../aclImdb/test/')

In [3]:
print(train_dataset)
print(train_dataset['labels'])
print(train_dataset.features)

Dataset({
    features: ['texts', 'labels'],
    num_rows: 25000
})
[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0

In [4]:
# 载入文本标记器
# cache_dir是预训练模型的地址
cache_dir="deberta-v3-base/"
tokenizer = AutoTokenizer.from_pretrained(cache_dir)

# 将数据转化为模型可以接受的格式
# 设置最大长度
MAX_LENGTH = 512

# 使用文本标记器对texts进行编码
train_dataset = train_dataset.map(lambda e: tokenizer(e['texts'], truncation=True, padding='max_length', max_length=MAX_LENGTH), batched=True)
test_dataset = test_dataset.map(lambda e: tokenizer(e['texts'], truncation=True, padding='max_length', max_length=MAX_LENGTH), batched=True)

# 将数据保存到本地
train_dataset.save_to_disk('./data/train_dataset')
test_dataset.save_to_disk('./data/test_dataset')


/home/xyzhang/.conda/envs/py39/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/25000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/25000 [00:00<?, ? examples/s]

In [5]:
print(train_dataset.features)

{'texts': Value(dtype='string', id=None), 'labels': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}


## 2 训练模型

In [1]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoConfig
import torch
from datasets import Dataset
import json
import os
# 设定使用的GPU编号，也可以不设置，但trainer会默认使用多GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
# 将num_labels设置为2，因为我们训练的任务为2分类
model = AutoModelForSequenceClassification.from_pretrained('deberta-v3-base/', num_labels=2)

# 加载处理好的数据
train_dataset = Dataset.load_from_disk('./data/train_dataset/')
test_dataset = Dataset.load_from_disk('./data/test_dataset/')
'''
这里可以冻结BERT参数，只训练最后一层二分类层，不过我这里采用的全部训练策略
for param in model.base_model.parameters():
    param.requires_grad = False
'''
# 训练超参配置
training_args = TrainingArguments(
    output_dir='./my_results',          # output directory 结果输出地址
    num_train_epochs=10,              # total # of training epochs 训练总批次
    per_device_train_batch_size=16,  # batch size per device during training 训练批大小
    per_device_eval_batch_size=16,   # batch size for evaluation 评估批大小
    logging_dir='./my_logs',            # directory for storing logs 日志存储位置
)
# batch_size 改为8
# 创建Trainer
trainer = Trainer(
    model=model.to('cuda'),              # the instantiated 🤗 Transformers model to be trained 需要训练的模型
    args=training_args,                  # training arguments, defined above 训练参数
    train_dataset=train_dataset,         # training dataset 训练集
    eval_dataset=test_dataset,           # evaluation dataset 测试集
)


/home/xyzhang/.conda/envs/py39/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at deberta-v3-base/ and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/xyzhang/.conda/envs/py39/lib/python3.9/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches',

In [4]:
# 开始训练
trainer.train()

# 开始评估模型
trainer.evaluate()

# 保存模型 会保存到配置的output_dir处
trainer.save_model()
torch.save(model.state_dict(), 'model_save.bin')

Step,Training Loss
500,0.276200
1000,0.225100
1500,0.203600
2000,0.133700
2500,0.147100
3000,0.138900
3500,0.102400
4000,0.082600
4500,0.088600
5000,0.075000


In [4]:
# 开始评估模型
trainer.evaluate()

{'eval_loss': 0.3597537577152252,
 'eval_runtime': 343.9999,
 'eval_samples_per_second': 72.674,
 'eval_steps_per_second': 4.544}

## 3 模型准确率评估

In [11]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DebertaConfig
import torch
from datasets import Dataset
import json
import os
import numpy as np
import csv
# Set the device to use the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 设定使用的GPU编号，也可以不设置，但trainer会默认使用多GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# 加载模型
output_config_file = './my_results/config.json'
output_model_file = 'model_save.bin'

config = AutoConfig.from_pretrained(output_config_file)
model = AutoModelForSequenceClassification.from_pretrained(output_model_file, config=config).to(device)


# 加载数据
test_dataset = Dataset.load_from_disk('./data/test_dataset/')
cache_dir="deberta-v3-base/"
tokenizer = AutoTokenizer.from_pretrained(cache_dir)

# 降低批处理大小和序列长度
batch_size = 16

data = test_dataset['texts']
data = tokenizer(data, max_length=512, truncation=True, padding='max_length', return_tensors="pt")

# 使用较小批次进行推理
preds = []
labels = []
for i in range(0, len(data['input_ids']), batch_size):
    batch_data = {k: v[i:i+batch_size].to(device) for k, v in data.items()}
    with torch.no_grad():
        batch_preds = model(**batch_data).logits
    preds.extend(np.argmax(batch_preds.detach().cpu().numpy(), axis=-1))
    labels.extend(np.argmax(test_dataset['labels'][i:i+batch_size], axis=-1))

preds = np.array(preds)
labels = np.array(labels)

# 计算准确率
accuracy = sum(preds == labels) / len(labels)
print(f"Accuracy: {accuracy:.2f}")

/home/xyzhang/.conda/envs/py39/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Accuracy: 0.95


In [ ]:
# 将一维数组转换为CSV文件中的单列
with open('output1.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(preds)  # 写入头部

# 将一维数组转换为CSV文件中的单列
with open('output2.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(labels)  # 写入头部

In [12]:
import csv
import numpy as np

# 读取 CSV 文件
with open('output1.csv', 'r') as csvfile1:
    reader1 = csv.reader(csvfile1)
    rows1 = list(reader1)

rows1 = np.array(rows1).astype(np.int32)
preds = rows1[0]

with open('output2.csv', 'r') as csvfile2:
    reader2 = csv.reader(csvfile2)
    rows2 = list(reader2)

rows2 = np.array(rows2).astype(np.int32)
labels = rows2[0]
# 计算准确率
accuracy = sum(preds == labels) / len(labels)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.95
